In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [12]:
#pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 
import pprint
glass = fetch_ucirepo(id=42)
X = glass.data.features #atributi
Y = glass.data.targets #vrednosti za celnata promenliva

pprint.pp(glass.metadata)

{'uci_id': 42,
 'name': 'Glass Identification',
 'repository_url': 'https://archive.ics.uci.edu/dataset/42/glass+identification',
 'data_url': 'https://archive.ics.uci.edu/static/public/42/data.csv',
 'abstract': 'From USA Forensic Science Service; 6 types of glass; defined in '
             'terms of their oxide content (i.e. Na, Fe, K, etc)',
 'area': 'Physics and Chemistry',
 'tasks': ['Classification'],
 'characteristics': ['Multivariate'],
 'num_instances': 214,
 'num_features': 9,
 'feature_types': ['Real'],
 'demographics': [],
 'target_col': ['Type_of_glass'],
 'index_col': ['Id_number'],
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 1987,
 'last_updated': 'Thu Aug 10 2023',
 'dataset_doi': '10.24432/C5WW2P',
 'creators': ['B. German'],
 'intro_paper': None,
 'additional_info': {'summary': 'Vina conducted a comparison test of her '
                                'rule-based system, BEAGLE, the '
                                'neare

In [13]:
X

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [14]:
Y

,Type_of_glass
0,1
1,1
2,1
3,1
4,1
...,...
209,7
210,7
211,7
212,7
